In [1]:
import numpy as np
import pydot
from IPython.display import display, HTML, SVG

from pydrake.all import (
    AddMultibodyPlantSceneGraph, DiagramBuilder, 
    FindResourceOrThrow, GenerateHtml, InverseDynamicsController, 
    MultibodyPlant, Parser, Simulator, MeshcatVisualizerCpp)
from manipulation import running_as_notebook
from manipulation.meshcat_cpp_utils import (
  StartMeshcat, MeshcatJointSliders)

In [2]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [3]:
import numpy as np

def joint_slider_demo(): 
  builder = DiagramBuilder()

  plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
    
  parser = Parser(plant, scene_graph)
  parser.package_map().PopulateFromEnvironment("ROS_PACKAGE_PATH")
  model = parser.AddModelFromFile("/home/avelino/catkin_ws/src/open_manipulator/open_manipulator_description/urdf/open_manipulator.urdf")

  plant.WeldFrames(plant.world_frame(), plant.get_body(
      plant.GetBodyIndices(model)[0]).body_frame())
  plant.Finalize()

  meshcat.Delete()
  meshcat.DeleteAddedControls()
  visualizer = MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)
  diagram = builder.Build()
  context = diagram.CreateDefaultContext()
  
  num = plant.num_positions()
  x = np.array(-10.)
  assert len(x.shape) <= 1
  return np.array(x) * np.ones(num)
        
  sliders = MeshcatJointSliders(meshcat, plant, visualizer, context)
  sliders.Run()

joint_slider_demo()


array([-10., -10., -10., -10., -10., -10.])

In [4]:
meshcat.Delete()
meshcat.DeleteAddedControls()

In [5]:
builder = DiagramBuilder()

# Adds both MultibodyPlant and the SceneGraph, and wires them together.
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)

parser = Parser(plant, scene_graph)
parser.package_map().PopulateFromEnvironment("ROS_PACKAGE_PATH")
model = parser.AddModelFromFile("/home/avelino/catkin_ws/src/open_manipulator/open_manipulator_description/urdf/open_manipulator.urdf")

plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("link1"))
plant.Finalize()

# Adds the MeshcatVisualizer and wires it to the SceneGraph.
visualizer = MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)

In [6]:
print(plant.num_positions())
print(plant.num_actuators())

6
6


In [7]:
Kp = np.full(6, 100)
Ki = 2 * np.sqrt(Kp)
Kd = np.full(6, 1)

robotis_controller = builder.AddSystem(InverseDynamicsController(plant, Kp, Ki, Kd, False))
robotis_controller.set_name("robotis_controller");
builder.Connect(plant.get_state_output_port(model),
                robotis_controller.get_input_port_estimated_state())
builder.Connect(robotis_controller.get_output_port_control(),
                plant.get_actuation_input_port())



In [8]:
diagram = builder.Build()
context = diagram.CreateDefaultContext()

In [9]:
plant_context = plant.GetMyMutableContextFromRoot(context)
#plant.get_actuation_input_port().FixValue(plant_context, np.zeros(6));

In [10]:
q0 = np.array([-1.57, 0.1, 0, -1.2, 0, 1.6])
x0 = np.hstack((q0, 0*q0))
plant.SetPositions(plant_context, q0)
robotis_controller.GetInputPort('desired_state').FixValue(
    robotis_controller.GetMyMutableContextFromRoot(context), x0)
print(context)

::_ Context (of a Diagram)
---------------------------
6 total continuous states
12 total discrete states in 1 groups
192 total numeric parameters in 29 groups
18 total abstract parameters

::_::plant Context
-------------------
Time: 0
States:
  1 discrete state groups with
     12 states
       -1.57   0.1     0  -1.2     0   1.6     0     0     0     0     0     0

Parameters:
  28 numeric parameter groups with
     1 parameters
       0
     1 parameters
       1
     1 parameters
       0
     1 parameters
       1
     1 parameters
       0
     1 parameters
       1
     1 parameters
       0
     1 parameters
       1
     1 parameters
       0
     1 parameters
       1
     1 parameters
       0
     1 parameters
       1
     10 parameters
       nan nan nan nan nan nan nan nan nan nan
     10 parameters
            0.07912  0.000308762            0 -0.000121765  0.000158069  0.000276884  0.000243616            0 -2.21913e-06            0
     10 parameters
          0.09840

In [11]:
simulator = Simulator(diagram, context)

In [12]:
simulator.set_target_realtime_rate(1.0)
simulator.AdvanceTo(5.0)